In [ ]:
workingdir=%pwd
%cd -q ../..

from __future__ import print_function

import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import yaml

from plotstyle import *
from df_collection import getDataframeCollection
from params import IsolationParams, ShowerShapeParams, ptcuttext, centralitycuttext
from template_fit import ExtendedTemplateFit
from utils import getHistAndErr, getNormHistAndErr, getCenters, getXerrForPlot

%cd $workingdir
plotdir = 'plots'
if not os.path.exists(plotdir):
    os.mkdir(plotdir)

In [ ]:
# load all YAML files
with open('config/globalconfig.yaml') as configfile:
    configgj = yaml.safe_load(configfile)
    
with open('config/systemconfig.yaml') as configfile:
    configsys = yaml.safe_load(configfile)

with open('config/runconfig.yaml') as configfile:
    configrun = yaml.safe_load(configfile)

configall = {}
configall.update(configgj)
configall.update(configsys)
configall.update(configrun)

centranges = [tuple(centrange) for centrange in configsys['centralityranges']]
ptranges = [tuple(ptrange) for ptrange in configsys['clusterptranges']]

if 'centralityvalues' in configsys['isolation']:
    centisoparams = {}
    isovar = configsys['isolation']['isovar']
    for params in configsys['isolation']['centralityvalues']:
        isoParams = IsolationParams(isovar)
        isoParams.setFromConfig(params)

        centrange = tuple(params['centralityrange'])
        centisoparams[centrange] = isoParams
else:
    isoParams = IsolationParams()
    isoParams.setFromConfig(configsys['isolation'])

ssParams = ShowerShapeParams()
ssParams.setFromConfig(configsys['showershape'])

# don't know why this doesn't get set when importing plotstyle, so do it again here
mpl.rc('figure', figsize=(8, 8))

In [ ]:
# print configuration for future reference
print(configall)

In [ ]:
# load CSVs; CSV directory is top-level
%cd -q ../..
print(datetime.datetime.now())
dfs = getDataframeCollection(configall)
%cd -q $workingdir

In [ ]:
# make all template fit plots and save purity values
purities = {}
purityerrs = {}

for centrange in centranges:
    isoParams = centisoparams[centrange]
    if centrange == (0, 10):
        trigCuts = ['isINT7 or isCentral']
    elif centrange == (30, 50):
        trigCuts = ['isINT7 or isSemiCentral']
    else:
        trigCuts = ['isINT7']
        
    additionalCuts = {}
    additionalCuts['gjmc'] = trigCuts
    additionalCuts['jjmc'] = trigCuts
    
    purities[centrange] = []
    purityerrs[centrange] = []
    
    for ptrange in ptranges:
        bkgWeights, _ = dfs.getBkgWeightsAndErrs(ptrange, isoParams, ssParams, centrange=centrange, useraa=True, additionalCuts=additionalCuts)
        tf = dfs.getTemplateFit(ptrange, isoParams, ssParams, bkgWeights=bkgWeights, centrange=centrange, additionalCuts=additionalCuts)
        purities[centrange].append(tf.purity)
        purityerrs[centrange].append(tf.purityerr)
        figfilename = '{0}/tf-{1}-{2}-{3}-{4}.png'.format(plotdir, centrange[0], centrange[1], ptrange[0], ptrange[1])
        tf.plotFitAndResiduals(ptrange, centrange, system=dfs.system, figfilename=figfilename)

In [ ]:
# print purity vs pT for all centralities
for centrange in centranges:
    print(centrange)
    print(purities[centrange])

In [ ]:
# plot purity vs pT for all centralities
def add10PercentBox(centrange):
    icentrange = centranges.index(centrange)
    color='C{0}'.format(icentrange)
    for ipt, ptrange in enumerate(ptranges):
        x = ptrange[0]
        width = ptrange[1] - ptrange[0]
        height = 0.2
        y = purities[centrange][ipt] - 0.1
        ax.add_patch(mpl.patches.Rectangle((x, y), width, height, color=color, alpha=0.15, ec='None'))
    plt.annotate('Shaded block shows\nextent of 10% purity\nvariation for {0}-{1}%'.format(*centrange),
                 xy=(0.97, 0.97), xycoords='axes fraction', ha='right', va='top', fontsize=20)

ptbincenters = getCenters(ptranges)
pterrs = getXerrForPlot(ptranges)

fig = plt.figure()
for centrange in centranges:
    plt.errorbar(ptbincenters, purities[centrange], yerr=purityerrs[centrange], xerr=pterrs, fmt='o',
                 label='{0}-{1}% ($p_\mathrm{{T}}^\mathrm{{iso}}$ < {2} GeV/$c$)'.format(centrange[0], centrange[1], centisoparams[centrange].isocut))

    ax = plt.gca()
    
# add10PercentBox(50, 90)

plt.legend(fontsize=18)
plt.ylabel('Purity (template fit)')
plt.xlabel('$p_\mathrm{T}^\mathrm{cluster}$ [GeV/$c$]')
plt.ylim([0.15, 0.75])
xlim = plt.xlim()
fig.savefig('{0}/purity-vs-pt.png'.format(plotdir))
plt.show()

### Get ABCD for comparison (Pb-Pb)
For the most direct comparison, use an isolation cut of 4 GeV/c and an anti-isolation range of 6--16 GeV/c and the 5x5all shower shape. Note that there may still be discrepancies.

In [ ]:
%cd -q ../..

import ROOT
from utils import th1ToArrays

th1Name = 'Purity_Iso3_Cone1'
centranges = [(0, 10), (10, 30), (30, 50), (50, 90)]

abcddata = {}

for icent, centrange in enumerate(centranges):
    arrs = {}
    
    rootfilename = 'abcd-plots/LHC15o_18qr_L1MB_EDMC_Cen{0}_JJRaaScaled.root'.format(icent)
    rootfile = ROOT.TFile.Open(rootfilename)
    purityth1 = rootfile.Get(th1Name)
    hist, err, centers, widths = th1ToArrays(purityth1)
    
    arrs['hist'] = hist
    arrs['err'] = err
    arrs['centers'] = centers
    arrs['widths'] = widths
    
    abcddata[centrange] = arrs
    rootfile.Close()
    
%cd -q $workingdir

In [ ]:
# plot ABCD purity vs pT for all centralities
ptbincenters = getCenters(ptranges)
pterrs = getXerrForPlot(ptranges)

fig = plt.figure()
for centrange in centranges:
    arrs = abcddata[centrange]
    gcenters = arrs['centers']
    ghist = arrs['hist']
    gerr = arrs['err']
    gwidths = arrs['widths']
    plt.errorbar(gcenters, ghist, yerr=gerr, xerr=np.divide(gwidths, 2), fmt='s', label='{0}-{1}%'.format(*centrange))
plt.legend()
plt.ylabel('Purity (ABCD)')
plt.xlabel('$p_\mathrm{T}^\mathrm{cluster}$ [GeV/$c$]')
plt.annotate('$p_\mathrm{T}^\mathrm{iso}$ < 4 GeV/$c$\n6-16 GeV/$c$',
             xy=(0.97, 0.03), xycoords='axes fraction', ha='right', va='bottom', fontsize=20)
plt.ylim([0.15, 0.75])
plt.xlim(xlim)
fig.savefig('{0}/purity-vs-pt-abcd-only.png'.format(plotdir))
plt.show()

In [ ]:
# plot ABCD and template fit purities together
ptbincenters = getCenters(ptranges)
pterrs = getXerrForPlot(ptranges)

fig = plt.figure()
for centrange in centranges:
    isoParams = centisoparams[centrange]
    plt.errorbar(ptbincenters, purities[centrange], yerr=purityerrs[centrange], xerr=pterrs, fmt='o',
                 label='{0}-{1}% ($p_\mathrm{{T}}^\mathrm{{iso}}$ < {2} GeV/$c$)'.format(centrange[0], centrange[1], isoParams.isocut))
plt.legend(fontsize=18)
plt.ylabel('Purity')
plt.xlabel('$p_\mathrm{T}^\mathrm{cluster}$ [GeV/$c$]')
upperLeftText('Faint markers are ABCD ($p_\mathrm{T}^\mathrm{iso}$ < 4 GeV/$c$)\nDark markers are template fit')

for irange, centrange in enumerate(centranges):
    arrs = abcddata[centrange]
    gcenters = arrs['centers']
    ghist = arrs['hist']
    gerr = arrs['err']
    gwidths = arrs['widths']
    plt.errorbar(gcenters, ghist, yerr=gerr, xerr=np.divide(gwidths, 2), fmt='C{0}s'.format(irange), alpha=0.6, mfc='white')

plt.ylim([0.15, 0.75])
plt.xlim(xlim)
fig.savefig('{0}/purity-vs-pt-abcd-comp.png'.format(plotdir))
plt.show()